# LangGraph Agent Test
Test ReAct agent with LangGraph BigTool

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
import os

In [ ]:
@tool
def calculate_integral_tool(function: str, x_min: float, x_max: float) -> str:
    """Calculate the definite integral of a mathematical function.
    
    Args:
        function: Mathematical function as string (e.g., 'x**2', 'sin(x)')
        x_min: Lower bound of integration
        x_max: Upper bound of integration
    
    Returns:
        String with the calculated integral value
    """
    import sympy as sp
    
    try:
        x = sp.Symbol('x')
        func = sp.sympify(function)
        result = sp.integrate(func, (x, x_min, x_max))
        return f"The integral of {function} from {x_min} to {x_max} is {float(result)}"
    except Exception as e:
        return f"Error calculating integral: {str(e)}"

# Test the tool directly
result = calculate_integral_tool.invoke({"function": "x**2", "x_min": 0, "x_max": 2})
print(result)

In [ ]:
# Create LLM and agent
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)
tools = [calculate_integral_tool]

agent = create_react_agent(llm, tools)

# Test agent
response = agent.invoke({
    "messages": [{
        "role": "user", 
        "content": "Calculate the integral of x^2 from 0 to 2 and explain the result"
    }]
})

print("Agent response:")
for message in response["messages"]:
    if message.type == "ai":
        print(f"AI: {message.content}")
    elif message.type == "tool":
        print(f"Tool: {message.content}")